# 

# Neo4j Graph Quickstart

## Connecting
Connect to the local neo4j container

In [1]:
import os
from dotenv import load_dotenv

load_dotenv("../.env")

# Defaults
NEO4J_USER = "neo4j" 
NEO4J_HOST = "127.0.0.1"

# Configurable
NEO4J_BROWSER_PORT = os.getenv("NEO4J_BROWSER_PORT")
NEO4J_BOLT_PORT = os.getenv("NEO4J_BOLT_PORT")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

In [2]:
from neo4j import GraphDatabase

uri = f"neo4j://{NEO4J_HOST}:{NEO4J_BOLT_PORT}"
driver = GraphDatabase.driver(uri, auth=(NEO4J_USER, NEO4J_PASSWORD))

driver.verify_connectivity()

## Clear Graph
Start this notebook with a blankslate

In [3]:
statement = """
MATCH (n) DETACH DELETE n
"""

results = driver.execute_query(statement)

## Create Nodes
Here is a data insert method

In [4]:
# Create the Nodes
statement = """
MERGE (:Person {name: 'Kenny'})
MERGE (:Person {name: 'Kyle'})
MERGE (:Person {name: 'Stan'})
MERGE (:Person {name: 'Eric'})
"""
_ = driver.execute_query(statement)

In [5]:
result = driver.execute_query("MATCH (n:Person) RETURN n.name as name")
for r in result.records:
    print(r['name'])

Kenny
Kyle
Stan
Eric


## Create Relationships
Kenny has become friends with Kyle

In [6]:
statement = """
MATCH (p1:Person {name: 'Kenny'})
MATCH (p2:Person {name: 'Kyle'})
MERGE (p1) -[:FRIEND]-> (p2)
"""
_ = driver.execute_query(statement)

In [7]:
result = driver.execute_query("MATCH (p1:Person) -[f:FRIEND]-> (p2:Person) RETURN p1.name, f, p2.name")
for p1, f, p2 in result.records:
    print(f"{p1} is a {f.type} of {p2}")

Kenny is a FRIEND of Kyle


## Updating
Kyle and Kenny got in a fight, they are now enemies

In [8]:
statement = """
MATCH (p1:Person {name: 'Kenny'}) -[f:FRIEND]-> (p2:Person {name: 'Kyle'})
DELETE f
MERGE (p1) -[:ENEMY]-> (p2)
"""
_ = driver.execute_query(statement)

In [9]:
result = driver.execute_query("MATCH (p1:Person) -[f]-> (p2:Person) RETURN p1.name, f, p2.name")
for p1, f, p2 in result.records:
    print(f"{p1} is an {f.type} of {p2}")

Kenny is an ENEMY of Kyle


## Vectors
Vectors are useful for semantic searches 

Here is a vector example using a euclidian distance

In [10]:
statement = """
MERGE (p1:Person {name: 'Kenny'}) SET p1.position = [1, 1]
MERGE (p2:Person {name: 'Kyle'}) SET p2.position = [2, 1]
MERGE (p3:Person {name: 'Stan'}) SET p3.position = [1, 2]
MERGE (p4:Person {name: 'Eric'}) SET p4.position = [3, 3]
"""
_ = driver.execute_query(statement)

In [11]:
statement = """
DROP INDEX position IF EXISTS
"""
_ = driver.execute_query(statement)

statement = """
CREATE VECTOR INDEX position
FOR (p:Person) ON p.position
OPTIONS {indexConfig: {
 `vector.dimensions`: 2,
 `vector.similarity_function`: 'euclidean'
}}
"""
_ = driver.execute_query(statement)

In [12]:
import math

statement = """
MATCH (p:Person {name: 'Kenny'})
CALL db.index.vector.queryNodes('position', 4, p.position)
YIELD node, score
WHERE node <> p
RETURN node.name as name, score
"""

result = driver.execute_query(statement)
for r in result.records:
    distance = math.sqrt(1/r['score']-1) # neo4j normalizes distance: 1/(1+distance)
    print(r['name'], round(distance, 1))

Stan 1.0
Kyle 1.0
Eric 2.8


## Traversal

In [13]:
statement = """
MATCH (p1:Person {name: 'Kenny'})
MATCH (p2:Person {name: 'Stan'})
MATCH (p3:Person {name: 'Kyle'})
MATCH (p4:Person {name: 'Eric'})

MERGE (p1)-[:FRIEND]->(p2)
MERGE (p2)-[:FRIEND]->(p3)
MERGE (p1)-[:FRIEND]->(p3)
MERGE (p3)-[:ENEMIES]->(p4)
MERGE (p4)-[:FRIEND]->(p1)
MERGE (p4)-[:FRIEND]->(p2)
"""
result = driver.execute_query(statement)

In [14]:
def get_all_networked_nodes(driver, name):
    statement = """
    MATCH (p:Person {name: $name})
    MATCH (p)-[f]->(n)
    RETURN n, f
    """
    result = driver.execute_query(statement, parameters_={'name': name})
    return result

name = 'Kenny'
print(name)
for i in range(3):
    for person in get_all_networked_nodes(driver, name).records:
        print(' '*(i*2) + f"- {person['n']['name']}: {person['f'].type}")
        name = person['n']['name']


Kenny
- Kyle: ENEMY
- Stan: FRIEND
- Kyle: FRIEND
  - Eric: ENEMIES
    - Kenny: FRIEND
    - Stan: FRIEND


## Visualizing
Neo4j offers a visualization tool here: http://127.0.0.1:7474/browser/

![](../assets/neo4j_example_visualization.png)

## Cleanup

In [15]:
driver.close()